In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
# import matplotlib.pyplot as plt
import pandas as pd

from utils.feature_loaders import load_feats
from utils.helper_fncs import load_json
from utils.db_utils import get_seq_names
# from run.pipeline import run_exp
from run.optim import run_for_set
from run.skopt_tune import TunerSkopt

from utils.feature_loaders import load_feats

from copy import deepcopy


Bad key "text.kerning_factor" on line 4 in
/home/korhan/miniconda3/envs/pytorch/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


This notebook shows how to perform parameter tuning with cross-validation.  
It uses `scikit-optimize` package to perform Bayesian parameter search. 

**sets**  
There are 9 signers in the Phoenix dataset, where the amount of data for each signer is highly imbalanced. Therefore the  
signers are grouped into 3 sets (named `A,B,C`) so that each set has similar amount of data. The assignments and the total  
number of sequences for each set are given in the cell below. 

**cross validation scheme**  
For each fold, perform parameter tune on devset and using those optimum parameters, report results on remaining test sets.  
Since test sets would contain different signers, this result can be viewed as test on unseen signers. At the end, compute  
a weighted average of the results for *dev* and *test* sets, weighted by total number of files in each set. Because the  
amount of data is very few, the test results are reported as average of cross validation rather than a single test set. 

| dev set |test sets|
|  :----: | :----: |
| A | B,C |  
| B | A,C |  
| C | A,B |


In [3]:
cvsets = set(['A','B','C'])
signers_per_set = {'A': [4,8,9], 'B':[2,5,7], 'C':[1,3,6]}
nfolder_perset = {'A':1705,'B':1991,'C':1975}

alg_type = 'knn'
params = load_json(full_path='./config/{}.json'.format(alg_type))
params

{'CVroot': '/home/korhan/Desktop/knn_utd/data/CVfolds',
 'CVset': 'A',
 'clustering': {'cost_thr': 0.01, 'method': 'pairwise', 'olapthr_m': 0.5},
 'disc_method': 'knn',
 'disc': {'a': 3,
  'dim_fix': 5,
  'emb_type': 'gauss_kernel',
  'k': 150,
  'lmax': 11,
  'lmin': 2,
  'metric': 'L2',
  'norm': False,
  'olapthr_m': 0.2,
  'pca': '',
  'r': 0.1,
  's': 0.4,
  'seg_type': 'uniform',
  'top_delta': 0.05,
  'use_gpu': True},
 'eval': {'tderunfile': '/home/korhan/Desktop/knn_utd/run_tde.sh',
  'TDEROOT': '/home/korhan/Desktop/tez/tdev2/tdev2',
  'TDESOURCE': '/home/korhan/miniconda3/etc/profile.d/conda.sh',
  'njobs': 2,
  'dataset': 'phoenix',
  'config_file': '/home/korhan/Desktop/knn_utd/config/config_phoenix.json'},
 'exp_root': '/home/korhan/Desktop/data/results/',
 'feats_root': '/home/korhan/Desktop/data/features/',
 'featype': 'c3right',
 'coverage': {'patience': 30, 'covth': 10, 'covmargin': 1},
 'tune': {'keys': ['lmax', 'dim_fix', 'r', 's'],
  'chkpoint_root': '/home/korhan/

In [4]:
# disable comment for choosing {openpose/deephand} features
# params['featype'] = 'op100'
params['featype'] = 'c3_right_PCA40W'

# parameters for skopt tuner
params['tune'] = {
        "keys": [ # list of discovery parameters to perform tuning, 
            "lmax", # the order should be the same as SPACE_KNN definition located in runexp/skopt_tune.py 
            "dim_fix",
            "r",
            "s"
        ],
        "chkpoint_root":"/home/korhan/Desktop/knn_utd/data/checkpoints", # the directory to save tuner checkpoints
        "minimizer": "gp", # type of skopt minimizer to use, {'rf': forest_minimize, 'gp': gp_minimize, 'gbrt': gbrt_minimize}
        "stop_margin": 0.01, # these two parameters are for early stop, if #patience best results are within margin
        "stopper_patience": 5
    }

params_default = deepcopy(params)


In [5]:


all_stats = dict() # store scores
tuners = dict() # store tuner objects for analysis afterwards

for devset in sorted(cvsets):    # for each fold
    # remaining sets are test sets
    testsets = sorted(cvsets - set(devset)) 
    
    # load related data
    params['CVset'] = devset
    seq_names = get_seq_names(params)
    feats_dict = load_feats(seq_names, params)
   
    # paramtune on devset
    print('=== Running Paramtune on Devset {} ==='.format(devset))
    tuner = TunerSkopt(seq_names, feats_dict, params)
    # set load_checkpoint True to continnue from previous tuning
    tuner.prepare(load_checkpoint=False) 
    xopt, score_min = tuner.run(n_calls=25)
    tuners[devset] = tuner
    print('=== Paramtune completed with min score {} ==='.format(score_min))
    
    # retrieve optimum parameters from tuner
    params = tuner.set_optim_params(params, xopt) 
    
    # re-run with tuned params to get full stats on dev + test sets
    stats = [run_for_set(setx, load_feats, params) for setx in sorted(cvsets)]
    
    all_stats[devset] = stats

    # re-set params
    params = deepcopy(params_default)

    # break

=== Running Paramtune on Devset A ===
Iteration No: 1 started. Searching for the next optimal point.
 === Running for new space {'lmax': 11, 'dim_fix': 5, 'r': 0.1, 's': 0.4} ===
knn_A_c3_right_PCA40W_a3_dim_fix5_emb_typegauss_kernel_k150_lmax11_lmin2_metricL2_normFalse_olapthr_m0.2_pca_r0.1_s0.4_seg_typeuniform_top_delta0.05_use_gpuTrue
0.01
*** Matches already discovered !!! ***
*** found 67518 matches ***
*** post disc completed, found 984 segments from 492 clusters ***
/home/korhan/miniconda3/etc/profile.d/conda.sh
/home/korhan/Desktop/data/results/knn_A_c3_right_PCA40W_a3_dim_fix5_emb_typegauss_kernel_k150_lmax11_lmin2_metricL2_normFalse_olapthr_m0.2_pca_r0.1_s0.4_seg_typeuniform_top_delta0.05_use_gpuTrue/postpairwise_cost0.01_olap0.5 phoenix sdtw /home/korhan/Desktop/data/results/knn_A_c3_right_PCA40W_a3_dim_fix5_emb_typegauss_kernel_k150_lmax11_lmin2_metricL2_normFalse_olapthr_m0.2_pca_r0.1_s0.4_seg_typeuniform_top_delta0.05_use_gpuTrue/postpairwise_cost0.01_olap0.5/scores.json


In [6]:
# type of scores to report results
report_cols = ['ned','length_avg', 'n_clus', 'n_node',
              'grouping_P', 'grouping_R', 'grouping_F', 
              'coverageNS']

# concat dataframes in 'all_stats' dict               
dflist = []
for devset, stats in all_stats.items():
    dflist.append(pd.DataFrame([{**{'devset':devset},**{'set':key}, **item} 
                                for sublist in stats for key,item in sublist.items()]))
    
df = pd.concat(dflist, ignore_index=True)
df['nfile'] = df.set.apply(lambda x: nfolder_perset[x])

df[report_cols] #.to_csv('../results/cv/{}_{}.csv'.format(params['disc_method'], params['featype']))

,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,coverageNS,nfile
0,38.43,13.422599,1185,2370,52.06,57.35,54.53,9.53,1705
1,47.14,15.335584,1366,2732,50.28,57.23,53.48,9.42,1991
2,41.48,15.069088,1203,2406,45.21,50.59,47.70,9.68,1975
3,46.62,12.594472,1262,2524,49.28,50.70,49.93,10.05,1705
4,43.88,13.946361,1360,2720,51.65,55.42,53.42,9.27,1991
5,46.81,13.125298,1321,2642,41.68,44.81,43.14,10.57,1975
6,41.45,12.257167,1281,2562,54.74,58.25,56.39,9.16,1705
7,42.34,15.356149,1573,3146,51.31,58.73,54.72,10.44,1991
8,42.25,14.237745,1202,2404,45.16,50.72,47.73,9.83,1975


In [7]:
# there are 3 sets of optimum params for 3 folds
df[params['disc'].keys()]

,a,dim_fix,emb_type,k,lmax,lmin,metric,norm,olapthr_m,pca,r,s,seg_type,top_delta,use_gpu
0,3,4,gauss_kernel,150,16,2,L2,False,0.5,,0.1847,0.05,uniform,0.05,True
1,3,4,gauss_kernel,150,16,2,L2,False,0.5,,0.1847,0.05,uniform,0.05,True
2,3,4,gauss_kernel,150,16,2,L2,False,0.5,,0.1847,0.05,uniform,0.05,True
3,3,8,gauss_kernel,150,14,2,L2,False,0.5,,0.2733,0.80,uniform,0.06,True
4,3,8,gauss_kernel,150,14,2,L2,False,0.5,,0.2733,0.80,uniform,0.05,True
5,3,8,gauss_kernel,150,14,2,L2,False,0.5,,0.2733,0.80,uniform,0.05,True
6,3,5,gauss_kernel,150,18,2,L2,False,0.5,,0.1488,0.05,uniform,0.06,True
7,3,5,gauss_kernel,150,18,2,L2,False,0.5,,0.1488,0.05,uniform,0.05,True
8,3,5,gauss_kernel,150,18,2,L2,False,0.5,,0.1488,0.05,uniform,0.05,True


seperate the results into two dataframes, for **devset** and **testset** results
for obtaining development and test scores. 


| devset | testset | df |
| :----: | :----:  | :----: |
| A | A | dev_df |
| B | B | dev_df |
| C | C | dev_df |
| A | B,C | test_df |
| B | A,C | test_df |
| C | A,B | test_df |


In [ ]:
devs_ms = []
tests_ms = []

for devset in ['A','B','C']:

    # seperate results into two different groups 
    fold_df = df.loc[df.devset.isin([devset])]
    dev_df = fold_df.loc[fold_df.set.isin([devset])][report_cols+['set']]
    test_df = fold_df.loc[~fold_df.set.isin([devset])][report_cols+['set']]

    # DEV
    fold = dev_df
    fold_ms = fold.loc[[len(x)==1 for x in fold.set]]
    devs_ms.append(fold_ms)
    
    # TEST
    fold = test_df
    fold_ms = fold.loc[[len(x)==1 for x in fold.set]]
    tests_ms.append(fold_ms)


: 

In [9]:
devs_ms[0]

,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,coverageNS,nfile,set
0,38.43,13.422599,1185,2370,52.06,57.35,54.53,9.53,1705,A


In [10]:
def averege(df_list, report_cols):
    ''' computes average scores weighted by number of files '''

    fold_df = pd.concat(df_list)
    tmp = pd.DataFrame(
        ((fold_df[report_cols].values * fold_df.nfile.values[:,None]).sum(0) / fold_df.nfile.sum())[None,:],
        columns=report_cols)
    return tmp



# compute the average scores weighted by number of files
keys = ['Dev MS'  ,'Test MS']

final = []

for k,df_list in enumerate([devs_ms , tests_ms]):
    avg_df = averege(df_list, report_cols)
    avg_df['exp'] = keys[k]
    
    final.append(avg_df)
    
    # display(avg_df)

finaldf = pd.concat(final)[['exp'] + report_cols]

finaldf

,exp,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,coverageNS,nfile
0,Dev MS,41.673775,13.890369,1252.360254,2504.720508,49.513040,54.363423,51.772107,9.543197,1899.441192
0,Test MS,44.320823,14.033080,1337.706225,2675.412449,48.600559,53.346069,50.794463,9.900199,1899.441192
